# IMA205 Challenge 2024 : Classify dermoscopic images among 8 different diagnostic classes

## Load Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import os
import cv2
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn_relief import ReliefF
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from skimage import measure
from skimage.measure import regionprops
from skimage import morphology
from skimage import exposure
from scipy.ndimage import zoom
from scipy.stats import gaussian_kde
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.losses import binary_crossentropy
from keras.metrics import MeanIoU
from keras.utils import to_categorical

## Load Data

In [ ]:
# meta data
meta_data_train = pd.read_csv('./ima205-challenge-2024/metadataTrain.csv')
meta_data_test = pd.read_csv('./ima205-challenge-2024/metadataTest.csv')

print('Meta data train:\n', meta_data_train)
print('Meta data test:\n', meta_data_test)

In [ ]:
# Example of how to see some particular column in the meta data 
meta_data_train['CLASS']

In [ ]:
## Extract images from Test and Train folders so that they match.
train_images = []
train_images_segmented = []
test_images = []
test_images_segmented = []

path_train = "./ima205-challenge-2024/Train/Train/"
path_test = "./ima205-challenge-2024/Test/Test/"

# Extract segmented images from the Train and Test folder and match them with the non-segmented images
content_train = os.listdir(path_train)
for i in range(len(content_train)):

  if content_train[i].lower().endswith(".png"):
    if content_train[i].strip("_seg.png") + ".jpg" in content_train:
      train_images_segmented.append(content_train[i])
      train_images.append(content_train[i-1])

content_test = os.listdir(path_test)
for i in range(len(content_test)):

  if content_test[i].lower().endswith(".png"):
    if content_test[i].strip("_seg.png") + ".jpg" in content_test:
      test_images_segmented.append(content_test[i])
      test_images.append(content_test[i-1])

print('Train images:\n', train_images)
print('Train images segmented:\n', train_images_segmented)

# Extract non-segmented images from the Train and Test folder
train_images_full = []
test_images_full = []
     
for i in range(len(content_train)):
  if content_train[i].lower().endswith(".jpg"):
    train_images_full.append(content_train[i])

for i in range(len(content_test)):
  if content_test[i].lower().endswith(".jpg"):
    test_images_full.append(content_test[i])

print('Train images full:\n', train_images_full)

In [ ]:
# Compute classes weights
class_weights = {}
for i in range(1, 9):
  class_weights[i] = len(meta_data_train[meta_data_train['CLASS'] == i]) / len(meta_data_train)

print('Class weights:\n', class_weights)

## Processing Method 1

### Preprocessing

In [ ]:
def dullRazor(image):
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY )

    #Black hat filter
    kernel = cv2.getStructuringElement(1,(9,9)) 
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)

    bhg = cv2.GaussianBlur(blackhat,(3,3),cv2.BORDER_DEFAULT)
    
    _,mask = cv2.threshold(bhg,10,255,cv2.THRESH_BINARY)

    #Replace pixels of the mask
    dst = cv2.inpaint(image,mask,6,cv2.INPAINT_TELEA)   

    return dst

In [ ]:
def crop_img(img, threshold=120):

    h, w = img.shape[:2]

    # Get the coordinates of the pixels in the diagonal
    y_coords = ([i for i in range(0, h, 3)], [i for i in range(h - 3, -1, -3)])
    x_coords = ([i for i in range(0, w, 4)], [i for i in range(0, w, 4)])

    # Get the mean value of the pixels in the diagonal, form 0,0 to h,w and from h,0 to 0,w
    coordinates = {'y1_1': 0, 'x1_1': 0, 'y2_1': h, 'x2_1': w, 'y1_2': h, 'x1_2': 0, 'y2_2': 0, 'x2_2': w}
    for i in range(2):
        d = []
        for y, x in zip(y_coords[i], x_coords[i]):
            d.append(np.mean(img[y, x, :]))

        # Get the location of the first point where the threshold is crossed
        for idx, value in enumerate(d):
            if value >= threshold:
                coordinates['y1_' + str(i + 1)] = y_coords[i][idx]
                coordinates['x1_' + str(i + 1)] = x_coords[i][idx]
                break

        # Get the location of the last point where the threshold is crossed
        for idx, value in enumerate(reversed(d)):
            if value >= threshold:
                coordinates['y2_' + str(i + 1)] = y_coords[i][-idx if idx != 0 else -1]
                coordinates['x2_' + str(i + 1)] = x_coords[i][-idx if idx != 0 else -1]
                break

    # Set the coordinates to crop the image
    y1 = max(coordinates['y1_1'], coordinates['y2_2'])
    y2 = min(coordinates['y2_1'], coordinates['y1_2'])
    x1 = max(coordinates['x1_1'], coordinates['x1_2'])
    x2 = min(coordinates['x2_1'], coordinates['x2_2'])

    if img[y1:y2, x1:x2, :].shape[0] < 20 or img[y1:y2, x1:x2, :].shape[1] < 20:
        return 0, h, 0, w, img
    
    return y1, y2, x1, x2, img[y1:y2, x1:x2, :]

In [ ]:
def preprocessing(img_data):
    try:
        # Crop the image to get the region of interest
        y1, y2, x1, x2, cropped_image = crop_img(img_data['original_image'])
        img_data['roi_coords'] = {'y1': y1, 'y2': y2, 'x1': x1, 'x2': x2}
        img_data['roi'] = cropped_image

        img_data['roi'] = dullRazor(img_data['roi'])

    except:
        img_data['roi_coords'] = {'y1': 0, 'y2': img_data['original_image'].shape[0], 'x1': 0, 'x2': img_data['original_image'].shape[1]}
        img_data['roi'] = img_data['original_image']

        img_data['roi'] = dullRazor(img_data['roi'])

    gray_image = cv2.cvtColor(img_data['roi'], cv2.COLOR_BGR2GRAY)

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_data['roi'] = clahe.apply(gray_image)

    # Apply median filtering
    img_data['roi'] = cv2.medianBlur(img_data['roi'], 5)

    return img_data

### Segmentation (Otsu)

In [ ]:
# Image segmentation
def segmentation(img_data):
    # otsu
    otsu_th, predicted_mask = cv2.threshold(img_data['roi'], 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    img_data['otsu_th'] = otsu_th
    img_data['mask_thresholding'] = predicted_mask

    return img_data

### Postprocessing

In [ ]:
def recover_original_mask_shape(original_shape, mask, roi_coords):
    original_mask = np.zeros(original_shape)

    for i, y in enumerate(range(roi_coords['y1'], roi_coords['y2'])):
        for j, x in enumerate(range(roi_coords['x1'], roi_coords['x2'])):
            original_mask[y, x] = mask[i, j]

    return original_mask

In [ ]:
def postprocessing(img_data):
    img_data['mask_thresholding'] = morphology.remove_small_holes(img_data['mask_thresholding'])

    image = morphology.remove_small_objects(img_data['mask_thresholding'], min_size=3500)

    if image.sum() == 0:
        img_data['mask_thresholding'] = morphology.remove_small_objects(img_data['mask_thresholding'], min_size=1000)
    else:
        img_data['mask_thresholding'] = image
    
    img_data['mask_thresholding'] = morphology.opening(img_data['mask_thresholding'], morphology.disk(12))
    img_data['mask_thresholding'] = morphology.dilation(img_data['mask_thresholding'], morphology.disk(12))

    img_data['mask_thresholding'] = morphology.convex_hull_image(img_data['mask_thresholding'], offset_coordinates=True)

    # Recover the original shape of the mask to match with image
    img_data['mask'] = recover_original_mask_shape(img_data['img_shape'], img_data['mask_thresholding'], img_data['roi_coords'])
    
    return img_data

### Results

In [ ]:
def skin_lesion_segmentation(img_root):
    # Load the image
    image = cv2.imread(img_root)

    # Dictionary to store the image and all the relevant information
    img_data = {}
    img_data['original_image'] = image
    img_data['img_shape'] = image.shape[:2]

    # Process the image
    img_data = preprocessing(img_data)
    img_data = segmentation(img_data)
    img_data = postprocessing(img_data)

    predicted_mask = img_data['mask']

    return predicted_mask

In [ ]:
# Example
for i in range(20,30):
    image = cv2.imread(path_train + train_images_full[i])
    mask = skin_lesion_segmentation(path_train + train_images_full[i])
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')

## Processing Method 2

### Preprocessing

In [ ]:
def dullRazor(image):
    grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY )

    #Black hat filter
    kernel = cv2.getStructuringElement(1,(9,9)) 
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)

    bhg = cv2.GaussianBlur(blackhat,(3,3),cv2.BORDER_DEFAULT)
    
    _,mask = cv2.threshold(bhg,10,255,cv2.THRESH_BINARY)

    #Replace pixels of the mask
    dst = cv2.inpaint(image,mask,6,cv2.INPAINT_TELEA)   

    return dst

In [ ]:
def preprocessing(image):

    image = dullRazor(image)

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    image = clahe.apply(gray_image)

    # Apply median filtering
    image = cv2.medianBlur(image, 5)

    return image

### Segmentation (U-Net)

In [ ]:
# Load images and masks as numpy arrays
X = []
Y = []
for i in range(len(train_images)):
    img = cv2.imread(path_train + train_images[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(path_train + train_images_segmented[i])

    # Preprocess the images and masks
    img = preprocessing(img)
    img = cv2.resize(img, (224, 224))
    img = np.reshape(img, (224, 224, 1))
    img = img / 255.0
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask = cv2.resize(mask, (224, 224))
    mask = np.reshape(mask, (224, 224, 1))
    mask = mask / 255.0

    X.append(img)
    Y.append(mask)

X = np.array(X)
Y = np.array(Y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Shuffle the training data
X_train, y_train = random.shuffle(X_train, y_train, random_state=42)

In [ ]:
# U-Net model
def unet(input_size = (224,224,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

# Create the U-Net model
model = unet()

# Fit the model
history = model.fit(X_train, y_train, batch_size=16, epochs=3, validation_data=(X_test, y_test))

In [ ]:
## Extract non segmented images from Test and Train folders
non_segmented_train_images = []
non_segmented_test_images = []

# Extract segmented images from the Train and Test folder and match them with the non-segmented images
content_train = os.listdir(path_train)
for i in range(len(content_train)):

  if content_train[i].lower().endswith(".png"):
    if not (content_train[i].strip("_seg.png") + ".jpg" in content_train):
        non_segmented_train_images.append(content_train[i])

content_test = os.listdir(path_test)
for i in range(len(content_test)):

  if content_test[i].lower().endswith(".png"):
    if not (content_test[i].strip("_seg.png") + ".jpg" in content_test):
        non_segmented_test_images.append(content_test[i])

In [ ]:
# Load images
X = []
for i in range(len(non_segmented_train_images)):
    img = non_segmented_train_images[i]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Preprocess the images and masks (e.g., resizing, normalization)
    img = preprocessing(img)
    img = cv2.resize(img, (224, 224))
    img = np.reshape(img, (224, 224, 1))
    img = img / 255.0

    X.append(img)

X = np.array(X)

# Make predictions
predictions_train = model.predict(X)

# Display the first image and its predicted mask
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(X[0].reshape(224, 224), cmap='gray')
plt.title("Image")
plt.subplot(122)
plt.imshow(predictions_train[0].reshape(224, 224), cmap='gray')
plt.title("Predicted Mask")
plt.show()

In [ ]:
# Load images
X = []
for i in range(len(non_segmented_test_images)):
    img = non_segmented_test_images[i]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Preprocess the images and masks (e.g., resizing, normalization)
    img = preprocessing(img)
    img = cv2.resize(img, (224, 224))
    img = np.reshape(img, (224, 224, 1))
    img = img / 255.0

    X.append(img)

X = np.array(X)

# Make predictions
predictions_test = model.predict(X)

# Display the first image and its predicted mask
plt.figure(figsize=(10, 10))
plt.subplot(121)
plt.imshow(X[0].reshape(224, 224), cmap='gray')
plt.title("Image")
plt.subplot(122)
plt.imshow(predictions_test[0].reshape(224, 224), cmap='gray')
plt.title("Predicted Mask")
plt.show()

### Postprocessing

In [ ]:
def postprocessing(mask):

    # Turn to binary mask
    mask = np.where(mask > 0.35, 1, 0)
    
    mask = morphology.remove_small_holes(mask)

    mask = morphology.remove_small_objects(mask, min_size=3500)

    if mask.sum() == 0:
        mask = morphology.remove_small_objects(mask, min_size=1000)
    else:
        mask = mask
    
    mask = morphology.opening(mask, morphology.disk(12))
    mask = morphology.dilation(mask, morphology.disk(12))

    return mask

## Feature Extraction

In [ ]:
# Function to compute asymmetry features
def compute_asymmetry_features(image, mask):
    # Find the center of mass of the lesion
    props = regionprops(mask)[0]
    center_y = int(props.centroid[0])
    center_x = int(props.centroid[1])

    # Define the symmetry axes
    angles = np.arange(0, 180, 10)

    S1_scores = []
    S2_scores = []

    for angle in angles:
        rotation_matrix = cv2.getRotationMatrix2D((center_x, center_y), angle, 1)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
        rotated_mask = cv2.warpAffine(mask, rotation_matrix, mask.shape[::-1], flags=cv2.INTER_NEAREST)

        A1 = rotated_mask[:center_y, :center_x]
        A2 = rotated_mask[:center_y, center_x:]
        A3 = rotated_mask[center_y:, :center_x]
        A4 = rotated_mask[center_y:, center_x:]

        # Compute the asymmetry scores
        S1 = np.sum(np.abs(rotated_image[A1].sum() + rotated_image[A2].sum() - (rotated_image[A3].sum() + rotated_image[A4].sum())))
        S2 = np.sum(np.abs(rotated_image[A1].sum() + rotated_image[A4].sum() - (rotated_image[A2].sum() + rotated_image[A3].sum())))

        # Divide by the area of the binary mask
        S1 /= np.sum(mask > 0)
        S2 /= np.sum(mask > 0)

        S1_scores.append(S1)
        S2_scores.append(S2)

    # The proposed asymmetry of shape features (f1, f2)
    f1 = np.min(S1_scores)
    f2 = np.min(S2_scores)

    return f1, f2

In [ ]:
# Function to compute border features
def compute_border_features(mask):
    # Compute the compactness of the lesion
    props = regionprops(mask)[0]
    perimeter = props.perimeter
    area = props.area
    compactness = perimeter ** 2 / (4 * np.pi * area)

    # Compute the fractal dimension of the binary mask
    fractal_dimension = 2 * np.log(perimeter) / np.log(area)

    f3, f4 = compactness, fractal_dimension
    
    return f3, f4

In [ ]:
# Function to compute color features
def compute_color_features(image, mask):
    # Convert RGB to CIE L*a*b* color space
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

   # Reshape the image to extract pixels
    pixels = image_lab.reshape(-1, image_lab.shape[-1])

    # Randomly select 10,000 pixels from the image
    num_pixels = 10000
    indices = random.sample(range(pixels.shape[0]), num_pixels)
    selected_pixels = pixels[indices]

    hist, _ = np.histogramdd(selected_pixels, bins=2, range=((0, 100), (-127, 127), (-127, 127)))

    # Compute the features
    f5 = np.mean(hist)
    f6 = np.var(hist)
    f7 = np.sum(hist > 0) / hist.size

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply the mask to the HSV image
    hsv_image = hsv_image * np.expand_dims(mask, axis=-1)

    # Split the HSV image into its channels
    h, s, v = cv2.split(hsv_image)

    # Define typical hue ranges for different colors
    white = [(0, 40), (220, 255)]
    red = [(160, 180)]
    light_brown = [(20, 25), (335, 360)]
    dark_brown = [(10, 15)]
    blue_gray = [(90, 120)]
    black = [(0, 20)]

    white_counter = 0
    red_counter = 0
    light_brown_counter = 0
    dark_brown_counter = 0
    blue_gray_counter = 0
    black_counter = 0

    # Iterate over each pixel in the hue channel
    for row in range(h.shape[0]):
        for col in range(h.shape[1]):
            hue = h[row, col]
            for color_range in [white, red, light_brown, dark_brown, blue_gray, black]:
                for range_ in color_range:
                    if range_[0] <= hue <= range_[1]:
                        if color_range == white:
                            white_counter += 1
                        elif color_range == red:
                            red_counter += 1
                        elif color_range == light_brown:
                            light_brown_counter += 1
                        elif color_range == dark_brown:
                            dark_brown_counter += 1
                        elif color_range == blue_gray:
                            blue_gray_counter += 1
                        elif color_range == black:
                            black_counter += 1

    f8, f9, f10, f11, f12, f13 = white_counter, red_counter, light_brown_counter, dark_brown_counter, blue_gray_counter, black_counter
    
    return f5, f6, f7, f8, f9, f10, f11, f12, f13

In [ ]:
# Function to compute geometric features
def compute_geometric_features(image, mask):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply binary thresholds at different percentiles
    thresholds = [np.percentile(gray_image[mask > 0], p) for p in [25, 50, 75]]
    binary_masks = (gray_image > thresholds[0]), (gray_image > thresholds[1]), (gray_image > thresholds[2])
    
    # Perform morphological opening to reduce noise
    disk = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opened_masks = [cv2.morphologyEx(mask, cv2.MORPH_OPEN, disk) & binary_mask for binary_mask in binary_masks]
    
    # Count the number of connected components (lesion pieces) in each mask
    f14, f15, f16 = [np.max(ndi.label(mask)[1]) for mask in opened_masks]

    return f14, f15, f16

In [ ]:
# Function to compute diameter feature
def compute_diameter_feature(mask):
    # Compute the diameter of the lesion
    props = regionprops(mask)[0]
    f17 = props.equivalent_diameter

    return f17

In [ ]:
def compute_features(image, mask):
    # Compute asymmetry features
    #f1, f2 = compute_asymmetry_features(image, mask)
    
    # Compute border features
    f3, f4 = compute_border_features(image, mask)
    
    # Compute color features
    f5, f6, f7, f8, f9, f10, f11, f12, f13 = compute_color_features(image, mask)
    
    # Compute geometric features
    f14, f15, f16 = compute_geometric_features(image, mask)
    
    # Compute diameter feature
    f17 = compute_diameter_feature(mask)
    
    # Concatenate all features
    features = np.array([f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17])
    
    return features

In [ ]:
df = pd.read_csv('./ima205-challenge-2024/metadataTrain.csv')
y_train = df['CLASS'].values
#class_names = ["Melanoma", "Melanocytic nevus", "Basal cell carcinoma", "Actinic keratosis", "Benign keratosis", "Dermatofibroma", "Vascular lesion", "Squamous cell carcinoma"]
X_train = np.zeros((len(df), 15))

for i in range(len(df)):
    image = cv2.imread('./ima205-challenge-2024/Train/Train/' + df['ID'][i] + '.jpg')
    image = cv2.resize(image, (224, 224))
    mask = cv2.imread('./ima205-challenge-2024/Train/Train/' + df['ID'][i] + '_seg.png', cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (224, 224))
    if mask is None:
        mask = skin_lesion_segmentation('./ima205-challenge-2024/Test/Test/' + df['ID'][i] + '.jpg')
        mask = cv2.resize(mask, (224, 224))
    mask = mask.astype(np.uint8)
    features = compute_features(image, mask)
    X_train[i] = features
    print(f'Processed image {i+1}/{len(df)}')

In [ ]:
df = pd.read_csv('./ima205-challenge-2024/metadataTest.csv')
X_test = np.zeros((len(df), 15))
for i in range(len(df)):
    image = cv2.imread('./ima205-challenge-2024/Test/Test/' + df['ID'][i] + '.jpg')
    image = cv2.resize(image, (224, 224))
    mask = cv2.imread('./ima205-challenge-2024/Test/Test/' + df['ID'][i] + '_seg.png', cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (224, 224))
    if mask is None:
        mask = skin_lesion_segmentation('./ima205-challenge-2024/Test/Test/' + df['ID'][i] + '.jpg')
        mask = cv2.resize(mask, (224, 224))
    mask = mask.astype(np.uint8)
    features = compute_features(image, mask)
    X_test[i] = features
    print(f'Processed image {i+1}/{len(df)}')

In [ ]:
# Un-used code

# Feature Selection with RELIEF Algorithm
def select_features(data, labels):
    # Apply RELIEF algorithm for feature selection
    selector = SelectKBest(score_func=ReliefF, k=10)
    selected_features = selector.fit_transform(data, labels)
    return selected_features

# Dimensionality Reduction with Linear PCA
def reduce_dimensionality(data, n_components):
    # Apply linear PCA for dimensionality reduction
    pca = PCA(n_components=n_components)
    reduced_data = pca.fit_transform(data)
    return reduced_data

# Asymmetry features
"""
def compute_asymmetry_features_unused(image, mask):
    # Find the center of mass of the lesion
    props = regionprops(mask)[0]
    center_y = int(props.centroid[0])
    center_x = int(props.centroid[1])

    # Define the symmetry axes
    angles = np.arange(0, 180, 10)

    # Initialize lists to store asymmetry scores
    S1_scores = []
    S2_scores = []
    C1_scores = []
    C2_scores = []

    for angle in angles:
        # Rotate the image and mask
        rotation_matrix = cv2.getRotationMatrix2D((center_x, center_y), angle, 1)
        rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
        rotated_mask = cv2.warpAffine(mask, rotation_matrix, mask.shape[::-1], flags=cv2.INTER_NEAREST)

        # Define the regions A1, A2, A3, and A4
        A1 = rotated_mask[:center_y, :center_x]
        A2 = rotated_mask[:center_y, center_x:]
        A3 = rotated_mask[center_y:, :center_x]
        A4 = rotated_mask[center_y:, center_x:]

        # Compute the asymmetry scores
        S1 = np.sum(np.abs(rotated_image[A1].sum() + rotated_image[A2].sum() - (rotated_image[A3].sum() + rotated_image[A4].sum())))
        S2 = np.sum(np.abs(rotated_image[A1].sum() + rotated_image[A4].sum() - (rotated_image[A2].sum() + rotated_image[A3].sum())))

        # Divide by the area of the binary mask
        S1 /= np.sum(mask > 0)
        S2 /= np.sum(mask > 0)

        S1_scores.append(S1)
        S2_scores.append(S2)

        # Combine regions for C1 and C2
        CA1_union_A2 = np.concatenate((A1.flatten(), A2.flatten()))
        CA3_union_A4 = np.concatenate((A3.flatten(), A4.flatten()))
        CA1_union_A4 = np.concatenate((A1.flatten(), A4.flatten()))
        CA2_union_A3 = np.concatenate((A2.flatten(), A3.flatten()))

        # Estimate the distribution of gray-scale values using Gaussian kernel density estimation
        kde = gaussian_kde(rotated_image.flatten())

        # Compute the asymmetry scores
        C1 = np.abs(kde.evaluate(CA1_union_A2) - kde.evaluate(CA3_union_A4)).mean()
        C2 = np.abs(kde.evaluate(CA1_union_A4) - kde.evaluate(CA2_union_A3)).mean()

        C1_scores.append(C1)
        C2_scores.append(C2)

    # The proposed asymmetry of shape features (f1, f2)
    f1 = np.min(S1_scores)
    f2 = np.min(S2_scores)

    # The proposed asymmetry of gray-scale features (f3, f4)
    f3 = np.min(C1_scores)
    f4 = np.min(C2_scores)

    # Calculate the radius of a circle with the same area as the lesion
    lesion_area = np.sum(mask > 0)
    equivalent_radius = np.sqrt(lesion_area / np.pi)
    n_distances = []

    # Generate alternative binary masks by applying thresholds at different percentiles
    thresholds = np.arange(0.1, 1, 0.1)
    for threshold in thresholds:
        thresholded_mask = (image > np.percentile(image[mask > 0], threshold * 100)) & mask
        xt, yt = regionprops(thresholded_mask)[0].centroid
        n_distances.append(np.linalg.norm([xt - center_x, yt - center_y]) / equivalent_radius)

    f5 = np.mean(n_distances)
    f6 = np.std(n_distances)

    return f1, f2, f3, f4, f5, f6
"""

# Border features
"""
def compute_border_features_unused(image, mask):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    border_pixels = mask ^ cv2.erode(mask, np.ones((3, 3), np.uint8))
    
    Rk_values = []
    
    # Sliding window around the border
    window_size = 61
    half_window = window_size // 2
    
    # Loop through each border pixel
    for k, (row, col) in enumerate(zip(*np.where(border_pixels == 1))):
        # Define the region around the border pixel
        roi = gray_image[row - half_window:row + half_window + 1, col - half_window:col + half_window + 1]
        
        # Extract pixels inside and outside the lesion
        X1 = roi[mask[row - half_window:row + half_window + 1, col - half_window:col + half_window + 1]]
        X2 = roi[~mask[row - half_window:row + half_window + 1, col - half_window:col + half_window + 1]]
        
        X1_mean = np.mean(X1)
        X2_mean = np.mean(X2)
        X_mean = np.mean(np.concatenate((X1, X2)))
        
        SST = np.sum((X1 - X_mean) ** 2) + np.sum((X2 - X_mean) ** 2)
        SSE = np.sum((X1 - X1_mean) ** 2) + np.sum((X2 - X2_mean) ** 2)
        Rk = SSE / SST
        Rk_values.append(Rk)
    
    f7 = np.percentile(Rk_values, 25)
    f8 = np.percentile(Rk_values, 50)
    f9 = np.percentile(Rk_values, 75)
    
    return f7, f8, f9
"""

# Color features
"""
# Convert RGB to CIE L*a*b* color space
image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

# Reshape the image to extract pixels
pixels = image_lab.reshape(-1, image_lab.shape[-1])

# Randomly select 10,000 pixels from the image
num_pixels = 10000
indices = random.sample(range(pixels.shape[0]), num_pixels)
selected_pixels = pixels[indices]

# Compute the 3D histogram with bin size n=2 The bin size is set to n = 2. Only the non-empty bins are considered for the score computation. We use the average number of samples in each bin (f10), the variance (f11),
# and the percentage of non-empty bins in the color space (f12). The L component values range from 0 to 100, while the a and b components vary between −127 and 127.
hist, _ = np.histogramdd(selected_pixels, bins=2, range=((0, 100), (-127, 127), (-127, 127)))

# Compute the features
f10 = np.mean(hist)
f11 = np.var(hist)
f12 = np.sum(hist > 0) / hist.size


# Divide the lesion area into inner and outer parts
inner_border = mask.copy()
iteration = 0
while np.sum(inner_border) / np.sum(mask) > 0.3:
    inner_border = zoom(inner_border, (1 - 0.05 * iteration, 1 - 0.05 * iteration, 1), order=0)
    iteration += 1

inner_mask = (mask * inner_border).astype(bool)
outer_mask = mask ^ inner_mask

# Compute the mean L*a*b* values for inner and outer parts
inner_lab = image_lab[inner_mask]
outer_lab = image_lab[outer_mask]

mean_inner_lab = np.mean(inner_lab, axis=0)
mean_outer_lab = np.mean(outer_lab, axis=0)

f13 = mean_inner_lab[0] - mean_outer_lab[0]
f14 = mean_inner_lab[1] - mean_outer_lab[1]
f15 = mean_inner_lab[2] - mean_outer_lab[2]

# Compute the probability density estimate of L*a*b* channels
kde_inner_lab = gaussian_kde(inner_lab.T)
kde_outer_lab = gaussian_kde(outer_lab.T)

# Compute the overlapping area of the densities for L*a*b* channels
f16 = np.abs(kde_inner_lab(mean_inner_lab) - kde_outer_lab(mean_inner_lab))
f17 = np.abs(kde_inner_lab(mean_inner_lab[0], mean_inner_lab[1]) - kde_outer_lab(mean_inner_lab[0], mean_inner_lab[1]))
f18 = np.abs(kde_inner_lab(mean_inner_lab[0], mean_inner_lab[2]) - kde_outer_lab(mean_inner_lab[0], mean_inner_lab[2]))
"""



## CNN

In [ ]:
X_train = np.array(X_train).reshape(-1, 1, 15, 1)
X_test = np.array(X_test).reshape(-1, 1, 15, 1)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=8)

In [ ]:
# Define the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1, 15, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=16, epochs=5, sample_weight=class_weights, validation_split=0.2)

In [ ]:
# Make predictions on the test set
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Print the predicted labels
print("Predicted labels:", predicted_labels)

In [ ]:
# Save the predictions to a CSV file
csv = pd.DataFrame({'ID': df['ID'], 'CLASS': predicted_labels})
csv.to_csv('sub1.csv', index=False)